In [38]:
import pandas as pd
import itertools
import numpy as np
import random
from random import choices

In [39]:
def ts_file_processor(file_name):
    with open (file_name,'r') as ts_file:
        ts=[]
        for i in ts_file:
            ts.append(i)
    ts=ts[0].split(',')
    ts=[float(ts[i]) for i in range(len(ts)) ]
    return ts

In [40]:
file='./original_series/original_series_'
names = locals()
ts_dict={}
for i in range(1,2001):
    ts_dict['ts_'+str(i)]=ts_file_processor(file+str(i)+'.txt')

In [41]:
ts_index=[i for i in range(1,2001)]

In [42]:
incremental_cd_df=pd.DataFrame({'start_point':[],
                               'ending_point':[],
                               'series':[]},
                                dtype=object)

In [43]:
def incremental_cd_simulation(no_simulation):
    for i in range(no_simulation):
        # choice two sereis from original series 
        series_index=np.random.choice(ts_index,2)
        ts1=ts_dict['ts_'+str(series_index[0])]
        ts2=ts_dict['ts_'+str(series_index[1])]
        
        incremental_cd_list=[]
        mixing_list=[]
        
        a=round(random.random()*len(ts1))
        b=round(random.random()*len(ts1))
        start_point,ending_point=sorted((a,b))
        
        incremental_cd_list.append(ts1[:start_point])
        
        ts_1_pick=ts1[start_point:ending_point]
        ts_2_pick=ts2[start_point:ending_point]
        
        weight=list(x for x in range(ending_point-start_point))
        weight=list(x/(ending_point-start_point) for x in weight)
        belta=list(1-i for i in weight)
        
        for n in range(ending_point-start_point):
            series=belta[n]*ts_1_pick[n]+weight[n]*ts_2_pick[n]
            mixing_list.append(series)
            
        incremental_cd_list.append(mixing_list)
            
        incremental_cd_list.append(ts2[ending_point:])
        
        incremental_cd_list=list(itertools.chain.from_iterable(incremental_cd_list))

        
        incremental_cd_df.loc[i]={
            'start_point':start_point,
            'ending_point':ending_point,
            'series':incremental_cd_list}
    return incremental_cd_df

In [44]:
incremental_cd_df=incremental_cd_simulation(2000)

,start_point,ending_point,series
0,1374,1531,"[3.62925351791304, 4.63594884063492, 3.2060850..."
1,1255,1838,"[4.38461249700816, 3.85387017513735, 3.8027997..."
2,1237,1901,"[4.23663643430782, 5.96075471478044, 4.7328381..."
3,1470,1534,"[4.60649951214697, 5.49004759491859, 4.4625278..."
4,193,877,"[4.37130669424373, 4.73872876298479, 5.0045263..."
...,...,...,...
1995,1484,1832,"[6.13679215719252, 2.89791814128522, 4.7072631..."
1996,1138,1364,"[1.12210585140928, 2.37182331676179, 2.2108695..."
1997,495,1572,"[3.56547779061701, 4.00779148447802, 4.2530695..."
1998,918,1835,"[3.30240587628332, 4.66984009402938, 4.7145785..."


In [45]:
incremental_cd_df.to_csv('./simulated_data/incremental_cd.csv',index=False)